# Data cleaning and concatination

In [110]:
import pandas as pd
import os
from tqdm import tqdm
import re
from dask import dataframe as dd
import csv
import lyricsgenius as lg

### Get all datafiles

### Lyrics cleaning function

In [3]:
def clean_lyrics(lyrics):
    lyrics = ' '.join(str(lyrics).splitlines()[:50])
    lyrics = ' '.join(str(lyrics).split()[:200])
    lyrics = re.sub("[\(\[\{].*?[\)\]\}]", "", lyrics)
    return lyrics

### ds2.csv (too big file)

In [16]:
dask_df = dd.read_csv(r"data//ds2.csv", on_bad_lines='skip', encoding='utf-8', quoting=csv.QUOTE_NONE, dtype={'views': 'object', 'year': 'object', 'lyrics': 'object'})

dask_df = dask_df.drop(['year', 'views', 'features', 'id'], axis=1)
dask_df = dask_df.dropna()
dask_df['lyrics'] = dask_df['lyrics'].apply(clean_lyrics, meta=('lyrics', 'str'))
# dask_df = dask_df.drop(dask_df[(dask_df['tag'] == 'misc') | (len(dask_df['lyrics']) < 200)])
dask_df.head()
    
dask_df.to_csv("data_cleaned/data1.csv", index=False)


c:\Users\zakamycze\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\io\csv.py:181: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\zakamycze\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\io\csv.py:181: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\zakamycze\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\io\csv.py:181: DtypeWarning: Columns (23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\zakamycze\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\io\csv.py:181: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\zakamycze\AppData\Local\Programs\Pytho

['c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\000.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\001.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\002.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\003.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\004.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\005.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\006.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\007.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\008.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\009.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\010.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\011.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\012.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\013.part',
 'c:\\Piotrek\\kod\\lyrics-to-generes\\data\\data1.csv\\014.pa

In [149]:
chunksize = 1_000_000
df = pd.read_csv(r"data//ds2.csv", nrows=chunksize)

df = df.drop(['year', 'views', 'features', 'id'], axis=1)
df = df.dropna()

df.rename(columns = {'tag': 'genre'}, inplace = True)

df = df.drop(df[(df['genre'] == 'misc') | (df['lyrics'] == "")].index)
df['lyrics'] = df['lyrics'].apply(clean_lyrics)
    
df.to_csv("data_cleaned/data1.csv", index=False)

### spotify_songs.csv

In [126]:
columns_to_read = ['track_name', 'track_artist', 'track_album_name', 'lyrics', 'playlist_subgenre']
df = pd.read_csv(r"data/spotify_songs.csv", usecols=columns_to_read)
df['lyrics'] = df['lyrics'].apply(clean_lyrics)
df.dropna()

df.rename(columns = {'track_name': 'track', 'track_artist':'artist', 'playlist_subgenre': 'genre', 'track_album_name':'album'}, inplace = True)
df = df.drop('album', axis=1)

unique_generes = df['genre'].unique()
print(len(unique_generes))
print(unique_generes)

df.head()

24
['classic rock' 'hard rock' 'new jack swing' 'neo soul' 'dance pop'
 'urban contemporary' 'big room' 'hip pop' 'latin pop' 'indie poptimism'
 'gangster rap' 'album rock' 'post-teen pop' 'trap' 'latin hip hop'
 'southern hip hop' 'tropical' 'electropop' 'progressive electro house'
 'pop edm' 'reggaeton' 'hip hop' 'permanent wave' 'electro house']


,track,artist,lyrics,genre
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,classic rock
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",hard rock
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",new jack swing
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,neo soul
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,dance pop


### test.csv

In [127]:
df2 = pd.read_csv(r"data/test.csv")
df2 = df2.drop(['Song year', 'Track_id'], axis=1)
df2.dropna()

df2.rename(columns = {'Song': 'track', 'Artist':'artist', 'Lyrics':'lyrics', 'Genre':'genre'}, inplace=True)

df2['lyrics'] = df2['lyrics'].apply(clean_lyrics)
df2['track'] = df2['track'].apply(lambda x: ' '.join(x.split('-')))
df2['artist'] = df2['artist'].apply(lambda x: ' '.join(x.split('-')))

unique_generes = df2['genre'].unique()
print(len(unique_generes))
print(unique_generes)

df2.head()

10
['Hip-Hop' 'Indie' 'Metal' 'Pop' 'Country' 'Jazz' 'Rock' 'R&B'
 'Electronic' 'Folk']


,track,artist,genre,lyrics
0,craftsmanship,buck 65,Hip-Hop,Most folks spend their days daydreaming of fin...
1,come on out,the elwins,Indie,Take your cold hands and put them on my face S...
2,riot,bullet for my valentine,Metal,Are you ready it's time for war We'll break do...
3,that s what girls do,dream street,Pop,You ask me why I change the color of my hair ...
4,believe in a dollar,cassidy,Hip-Hop,Do you believe in magic in a young girl's hear...


### train.csv

In [125]:
df3 = pd.read_csv(r"data/train.csv")
df3.dropna()

df3.rename(columns = {'Song': 'track', 'Artist':'artist', 'Lyrics':'lyrics', 'Genre':'genre', 'Language': 'language'}, inplace=True)

df3['lyrics'] = df3['lyrics'].apply(clean_lyrics)
df3 = df3.drop(df3[df3['language'] != 'en'].index)
df3 = df3.drop('language', axis=1)

unique_generes = df3['genre'].unique()
print(len(unique_generes))
print(unique_generes)

df3.head()

10
['Rock' 'Metal' 'Pop' 'Indie' 'Folk' 'Electronic' 'R&B' 'Jazz' 'Hip-Hop'
 'Country']


,artist,track,genre,lyrics
0,12 stones,world so cold,Rock,"It starts with pain, followed by hate Fueled b..."
1,12 stones,broken,Rock,Freedom! Alone again again alone Patiently wai...
2,12 stones,3 leaf loser,Rock,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,You say you know just who I am But you can't i...
4,12 stones,adrenaline,Rock,My heart is beating faster can't control these...


In [132]:
dff = pd.concat([df, df2, df3], ignore_index=True)

unique_generes = dff['genre'].unique()
print(len(unique_generes))
print(unique_generes)

print(len(dff.index))

dff.to_csv(r"data_cleaned/data4.csv", index=False)

dff.head()

34
['classic rock' 'hard rock' 'new jack swing' 'neo soul' 'dance pop'
 'urban contemporary' 'big room' 'hip pop' 'latin pop' 'indie poptimism'
 'gangster rap' 'album rock' 'post-teen pop' 'trap' 'latin hip hop'
 'southern hip hop' 'tropical' 'electropop' 'progressive electro house'
 'pop edm' 'reggaeton' 'hip hop' 'permanent wave' 'electro house'
 'Hip-Hop' 'Indie' 'Metal' 'Pop' 'Country' 'Jazz' 'Rock' 'R&B'
 'Electronic' 'Folk']
276586


,track,artist,lyrics,genre
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,classic rock
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",hard rock
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",new jack swing
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,neo soul
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,dance pop


### val2.csv

In [134]:
df4 = pd.read_csv(r"data/val2.csv")
df4.dropna()

df4.rename(columns = {'Lyric':'lyrics', 'Genre0':'genre'}, inplace=True)

df4['lyrics'] = df4['lyrics'].apply(clean_lyrics)

unique_generes = df4['genre'].unique()
print(len(unique_generes))
print(unique_generes)

print(len(df4.index))

df4.head()

6
['Rock' 'Hip Hop' 'Heavy Metal' 'Pop' 'Indie' 'Dance']
5389


,lyrics,genre
0,I was nineteen when I came to town they called...,Rock
1,Your coat and hat are gone. I've really can't ...,Rock
2,Use my Body to keep you with mine. Yeah I'm on...,Hip Hop
3,I met her at the 'Burger King'. We fell in lov...,Rock
4,This is the end of everything. A degenerate cu...,Heavy Metal


### lyrics-data.csv

In [135]:
df5 = pd.read_csv(r"data/lyrics-data.csv")
df5.dropna()

df5.rename(columns = {'Lyric':'lyrics', 'Genre0':'genre', 'SName': 'track', 'ALink':'link'}, inplace=True)

df5['lyrics'] = df5['lyrics'].apply(clean_lyrics)
df5 = df5.drop(df5[df5['language'] != 'en'].index)
df5 = df5.drop(['language', 'SLink'], axis=1)

print(len(df5.index))
df5.head()

191814


,link,track,lyrics
69,/ivete-sangalo/,Careless Whisper,I feel so unsure As I take your hand and lead ...
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,"Don't let them fool, ya Or even try to school,..."
88,/ivete-sangalo/,Cruisin' (Part. Saulo),"Baby, let's cruise, away from here Don't be co..."
111,/ivete-sangalo/,Easy,"Know it sounds funny But, I just can't stand t..."
140,/ivete-sangalo/,For Your Babies (The Voice cover),You've got that look again The one I hoped I h...


### artists-data.csv

In [136]:
df6 = pd.read_csv(r"data/artists-data.csv")
df6.dropna()

df6.rename(columns = {'Artist':'artist', 'Genres':'genre', 'Link':'link'}, inplace=True)

df6 = df6.drop(['Popularity', 'Songs'], axis=1)

unique_generes = df6['genre'].unique()
print(len(unique_generes))
print(unique_generes)

print(len(df6.index))
df6.head()

1751
['Pop; Axé; Romântico' 'Axé' 'Axé; Romântico; Reggae' ...
 'World Music; Indie; New Age' 'World Music; Gospel/Religioso'
 'World Music; Black Music; Blues']
4168


,artist,genre,link
0,Ivete Sangalo,Pop; Axé; Romântico,/ivete-sangalo/
1,Chiclete com Banana,Axé,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,/banda-eva/
3,É O Tchan,Axé,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,/claudia-leitte/


### join artist-data with lyrics-data

In [137]:
df7 = pd.merge(df5, df6, on='link')

df7 = df7.drop(['link'], axis=1)

df7.to_csv(r"data_cleaned/data5.csv", index=False)

print(len(df7.index))
df7.head()

191387


,track,lyrics,artist,genre
0,Careless Whisper,I feel so unsure As I take your hand and lead ...,Ivete Sangalo,Pop; Axé; Romântico
1,Could You Be Loved / Citação Musical do Rap: S...,"Don't let them fool, ya Or even try to school,...",Ivete Sangalo,Pop; Axé; Romântico
2,Cruisin' (Part. Saulo),"Baby, let's cruise, away from here Don't be co...",Ivete Sangalo,Pop; Axé; Romântico
3,Easy,"Know it sounds funny But, I just can't stand t...",Ivete Sangalo,Pop; Axé; Romântico
4,For Your Babies (The Voice cover),You've got that look again The one I hoped I h...,Ivete Sangalo,Pop; Axé; Romântico


### multifile_data

In [105]:
a = pd.read_csv(r"data/multifile_data/id_genres.csv", delimiter='\t')
b = pd.read_csv(r"data/multifile_data/id_information.csv", delimiter='\t')
c = pd.read_csv(r"data/multifile_data/id_genres.csv", delimiter='\t')

df8 = pd.merge(a, b, on='id')
df8 = pd.merge(df8, c, on='id')

df8.rename(columns = {'album_name':'album', 'song':'track', 'genres_y':'genre'}, inplace=True)

df8 = df8.drop(['id', 'genres_x'], axis=1)

unique_generes = df8['genre'].unique()
print(len(unique_generes))
print(unique_generes)

df8.to_csv(r"data_cleaned/data2.csv", index=False)

print(len(df8.index))
df8.head()

23520
['pop' 'underground hip hop' 'hard rock,rock,classic rock' ...
 'electropop,house,electronic' 'j-pop,folk,pop,soundtrack'
 'synthpop,remix,pop']
109269


,artist,track,album,genre
0,Cheryl,Rain on Me,3 Words,pop
1,Oddisee,After Thoughts,The Beauty in All,underground hip hop
2,Blue Öyster Cult,ME 262,Secret Treaties,"hard rock,rock,classic rock"
3,Rhapsody,Flames of Revenge,Legendary Years (Re-Recorded),"symphonic metal,power metal,symphonic power metal"
4,The Chameleons,Nostalgia,What Does Anything Mean? Basically (2009 Remas...,"post-punk,new wave"


### Merging cleaned data

In [179]:
a = pd.read_csv(r"data_cleaned/data1.csv")
b = pd.read_csv(r"data_cleaned/data4.csv")
c = pd.read_csv(r"data_cleaned/data5.csv")

print(len(a.index))
print(len(b.index))
print(len(c.index))

907893
276586
191387


In [180]:
a.head()

,title,genre,artist,lyrics
0,Killa Cam,rap,Cam'ron,"Killa Cam, Killa Cam, Cam Killa Cam, Killa Ca..."
1,Can I Live,rap,JAY-Z,"Yeah, hah, yeah, Roc-A-Fella We invite you t..."
2,Forgive Me Father,rap,Fabolous,Maybe cause I'm eatin And these bastards fiend...
3,Down and Out,rap,Cam'ron,"Ugh, Killa! Baby! Kanye, this that 1970s Her..."
4,Fly In,rap,Lil Wayne,"So they ask me ""Young boy What you gon' do th..."


In [181]:
b.head()

,track,artist,lyrics,genre
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,classic rock
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",hard rock
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",new jack swing
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,neo soul
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,dance pop


In [182]:
c.head()

,track,lyrics,artist,genre
0,Careless Whisper,I feel so unsure As I take your hand and lead ...,Ivete Sangalo,Pop; Axé; Romântico
1,Could You Be Loved / Citação Musical do Rap: S...,"Don't let them fool, ya Or even try to school,...",Ivete Sangalo,Pop; Axé; Romântico
2,Cruisin' (Part. Saulo),"Baby, let's cruise, away from here Don't be co...",Ivete Sangalo,Pop; Axé; Romântico
3,Easy,"Know it sounds funny But, I just can't stand t...",Ivete Sangalo,Pop; Axé; Romântico
4,For Your Babies (The Voice cover),You've got that look again The one I hoped I h...,Ivete Sangalo,Pop; Axé; Romântico


In [193]:
a.rename(columns = {'title':'track'}, inplace=True)

df = pd.concat([a, b, c], ignore_index=True)

genres = df['genre'].unique()
unique_genres = set()

for i in genres:
    if type(i) != str:
        continue
    for k in i.split(';'):
        unique_genres.add(k)

print(len(unique_generes))
print(unique_generes)

print(len(df.index))
df.head()

1223
['rap' 'rb' 'rock' ... 'World Music; Indie; New Age'
 'World Music; Gospel/Religioso' 'World Music; Black Music; Blues']
1375866


,track,genre,artist,lyrics
0,Killa Cam,rap,Cam'ron,"Killa Cam, Killa Cam, Cam Killa Cam, Killa Ca..."
1,Can I Live,rap,JAY-Z,"Yeah, hah, yeah, Roc-A-Fella We invite you t..."
2,Forgive Me Father,rap,Fabolous,Maybe cause I'm eatin And these bastards fiend...
3,Down and Out,rap,Cam'ron,"Ugh, Killa! Baby! Kanye, this that 1970s Her..."
4,Fly In,rap,Lil Wayne,"So they ask me ""Young boy What you gon' do th..."


In [199]:
change = {
    'rb': 'r&b',
    'hip hop': 'hip-hop',
    'hip pop': 'hip-hop',
    'album rock': 'rock',
    'post-teen pop': 'pop',
    'indie poptimism': 'indie;pop',
    'latin hip hop': 'latin hip-hop',
    'southern hip hop': 'southern hip-hop',
    'latin hip hop': 'latin hip-hop',
    'pop edm': 'pop;edm'
}


def genre_clean(genre):
    if genre in change.keys():
        return change[genre]
    else:
        return str(genre).lower().replace("; ", ";").replace(r"/", ";")


print(len(df.index))

df['genre'] = df['genre'].apply(genre_clean)
df = df.drop(df[(df.genre == "") | (df.genre == 'nan') | (df.lyrics == "")].index)

genres = df['genre'].unique()
unique_genres2 = set()

for i in genres:
    if type(i) != str:
        continue
    for k in i.split(';'):
        unique_genres2.add(genre_clean(k))

print(len(unique_genres2))
print(unique_genres2)

print(len(df.index))
df.head()

1375857
93
{'rock', 'dance pop', 'new jack swing', 'tropical house', 'metal', 'folk', 'clássico', 'hardcore', 'mpb', 'forró', 'r&b', 'instrumental', 'emocore', 'religioso', 'hip-hop', 'hard rock', 'electropop', 'regional', 'heavy metal', 'romântico', 'dance', 'rockabilly', 'gótico', 'reggaeton', 'axé', 'grunge', 'lo-fi', 'big room', 'velha guarda', 'infantil', 'blues', 'tecnopop', 'psicodelia', 'progressivo', 'permanent wave', 'k-rock', 'pós-punk', 'surf music', 'piano rock', 'bossa nova', 'trap', 'pagode', 'j-pop', 'world music', 'jovem guarda', 'trip-hop', 'house', 'southern hip-hop', 'industrial', 'ska', 'electronica', 'trance', 'rock alternativo', 'edm', 'funk', 'new wave', 'country', 'classic rock', 'post-rock', 'power-pop', 'j-rock', 'electro house', 'trilha sonora', 'jazz', 'chillout', 'soft rock', 'kizomba', 'k-pop', 'coletânea', 'black music', 'fado', 'pop', 'electronic', 'progressive electro house', 'neo soul', 'punk', 'indie', 'disco', 'soul music', 'latin pop', 'reggae', 'u

,track,genre,artist,lyrics
0,Killa Cam,rap,Cam'ron,"Killa Cam, Killa Cam, Cam Killa Cam, Killa Ca..."
1,Can I Live,rap,JAY-Z,"Yeah, hah, yeah, Roc-A-Fella We invite you t..."
2,Forgive Me Father,rap,Fabolous,Maybe cause I'm eatin And these bastards fiend...
3,Down and Out,rap,Cam'ron,"Ugh, Killa! Baby! Kanye, this that 1970s Her..."
4,Fly In,rap,Lil Wayne,"So they ask me ""Young boy What you gon' do th..."


In [200]:
df.to_csv(r"data_cleaned/data_final.csv", index=False)